In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import copy
import json
import openai
from dotenv import load_dotenv

from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from method.models import generate_random_values, generate_static_values, generate_llm_values
from method.ours.utils import create_driver, get_xpath, interact_with_input
from method.ours.history import HistoryTable
from method.ours.feedback import get_global_feedback

In [3]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Global Variables
HEADLESS = False
TEXT_EMBEDDING_METHOD = 'ADA' # ['ADA', 'WORD2VEC', 'SPACY']
GRAPH_EMBEDDING_METHOD = 'NODE2VEC' # ['NODE2VEC', 'GCN']

URL = 'https://www.united.com/en/us'
#'https://www.mbta.com/'
# 'https://www.united.com/en/us'
# 'https://www.uhaul.com/Truck-Rentals/'
# 'http://localhost:8080/1/common/items/create'
# 'https://www.cars.com/'
# 'https://www.ups.com/ca/en/Home.page'
# 'https://www.healthgrades.com/find-a-doctor'
# 'https://www.babycenter.com/pregnancy-weight-gain-estimator'
# 'https://www.budget.com/en/home'
# 'https://www.yelp.com/'
# 'https://www.aircanada.com/ca/en/aco/home.html'

In [4]:
METHOD = 'llm'

In [5]:
def get_to_form(driver):
    try:
        driver.get(URL)
    except:
        print('timeout')

    '''
    time.sleep(2)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/DIV[1]/DIV[2]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]')
        )
    ).click()
    '''
    
    '''
    try:
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.ID,
                'email')
            )
        ).send_keys('me@company.com')
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.ID,
                'password')
            )
        ).send_keys('password')
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//*[@type="submit"]')
            )
        ).click()
    except:
        pass
    '''
    
    '''
    # Pet Clinic - Add Owner
    time.sleep(2)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[2]')
        )
    ).click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[2]/UL[1]/LI[2]')
        )
    ).click()
    '''
    
    '''
    # UPS - Quote
    time.sleep(2)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'tabs_0_tab_1'))
    ).click()
    '''
    
    '''
    # AC - Multi-city
    time.sleep(2)
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, 'bkmgFlights_tripTypeSelector_M'))
    ).click()
    '''
    
    '''
    # AC - My Bookings
    time.sleep(2)
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, 'bkmg-tab-button-mngBook'))
    ).click()'''
    
    '''
    # Budget - Reservation
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, 'PicLoc_value'))
    ).click()
    '''

In [6]:
def find_form():
    return WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/DIV[1]/DIV[1]/DIV[1]/DIV[1]/DIV[3]/DIV[2]/DIV[2]/DIV[1]/DIV[1]/DIV[1]/MAIN[1]/SECTION[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/SECTION[1]/DIV[1]/DIV[1]/DIV[2]/FORM[1]'
        ))
    )


def find_button():
    # return find_form().find_elements(By.TAG_NAME, 'button')[1]
    return None


In [7]:
driver = create_driver(HEADLESS)
get_to_form(driver)

In [8]:
html = driver.find_element(By.TAG_NAME, 'html').get_attribute('outerHTML')
form = find_form()
form_xpath = get_xpath(driver, form)

inputs = form.find_elements(
    By.XPATH,
    f'{form_xpath}//input | {form_xpath}//textarea | {form_xpath}//select'
)

inputs = list(filter(
    lambda x: x.get_attribute('type') != 'hidden' and x.get_attribute('hidden') != 'true',
    inputs
))

In [9]:
history_table = HistoryTable(
    url=URL,
    xpath=form_xpath
)

In [10]:
values = None
if METHOD == 'static':
    values = generate_static_values(driver, inputs)
elif METHOD == 'random':
    values = generate_random_values(driver, inputs)
else:
    values = generate_llm_values(form.get_attribute('outerHTML'), openai.api_key)

  Prompt: [
  {
    "role": "system",
    "content": "\nYour task is to generate testing values for a web form. When given a web form, you need to generate values that can pass and fail when tried on the form. Your response should be in the following format:\n{\n[field_identifier]: {\npassing: <a base passing value for the field>,\nfailing: <a list of failing values for the field>\n}\n}\nYour response must be parsable by Python's json.loads function. Use \"id\" attribute to refer to the fields.\n"
  },
  {
    "role": "user",
    "content": "<form id=\"bookFlightForm\" class=\"app-components-BookFlightForm-bookFlightForm__bookFlightForm--2KgqV\"><div class=\"app-components-BookFlightForm-bookFlightForm__formHeader--2Hpvk\"><fieldset name=\"flightType\" class=\"app-components-BookFlightForm-bookFlightForm__radioFieldGroup--3IMzX\"><div class=\"app-components-BookFlightForm-bookFlightForm__radioContainer--3WXQM\"><label for=\"roundtrip\" class=\"app-components-BookFlightForm-bookFlightFo

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "{\n\"roundtrip\": {\n\"passing\": true,\n\"failing\": [false, \"true\", \"false\", 1, 0, \"1\", \"0\", null]\n},\n\"oneway\": {\n\"passing\": false,\n\"failing\": [true, \"true\", \"false\", 1, 0, \"1\", \"0\", null]\n},\n\"award\": {\n\"passing\": false,\n\"failing\": [true, \"true\", \"false\", 1, 0, \"1\", \"0\", null]\n},\n\"flexibleDates\": {\n\"passing\": false,\n\"failing\": [true, \"true\", \"false\", 1, 0, \"1\", \"0\", null]\n},\n\"bookFlightOriginInput\": {\n\"passing\": \"Vancouver YVR\",\n\"failing\": [\"\", \"123\", \"Vancouver\", \"YVR\", \"Vancouver YVR Vancouver YVR\", null]\n},\n\"bookFlightDestinationInput\": {\n\"passing\": \"Toronto YYZ\",\n\"failing\": [\"\", \"123\", \"Toronto\", \"YYZ\", \"Toronto YYZ Toronto YYZ\", null]\n},\n\"DepartDate\": {\n\"passing\": \"Sep 22\",\n\"failing\": [\"\", \"22 Sep\", \"September 22\", \"22/09\", \"22-09\",

In [11]:
values

{'roundtrip': {'passing': True,
  'failing': [False, 'true', 'false', 1, 0, '1', '0', None]},
 'oneway': {'passing': False,
  'failing': [True, 'true', 'false', 1, 0, '1', '0', None]},
 'award': {'passing': False,
  'failing': [True, 'true', 'false', 1, 0, '1', '0', None]},
 'flexibleDates': {'passing': False,
  'failing': [True, 'true', 'false', 1, 0, '1', '0', None]},
 'bookFlightOriginInput': {'passing': 'Vancouver YVR',
  'failing': ['',
   '123',
   'Vancouver',
   'YVR',
   'Vancouver YVR Vancouver YVR',
   None]},
 'bookFlightDestinationInput': {'passing': 'Toronto YYZ',
  'failing': ['', '123', 'Toronto', 'YYZ', 'Toronto YYZ Toronto YYZ', None]},
 'DepartDate': {'passing': 'Sep 22',
  'failing': ['',
   '22 Sep',
   'September 22',
   '22/09',
   '22-09',
   '2022-09-22',
   None]},
 'ReturnDate': {'passing': 'Sep 24',
  'failing': ['',
   '24 Sep',
   'September 24',
   '24/09',
   '24-09',
   '2022-09-24',
   None]},
 'passengerSelector': {'passing': '1 Adult',
  'failing': [

In [12]:
def get_element(element_id):
    return driver.find_element(By.ID if METHOD == 'llm' else By.XPATH, element_id)


def send_values(elemend_ids, values):
    for element_id in elemend_ids:
        try:
            element = get_element(element_id)
        except:
            continue
        value = values[element_id]
        
        element_type = element.get_attribute('type') or 'text'
    
        if element_type in ['radio', 'checkbox'] or element.tag_name in ['button']:
            continue
        
        try:
            interact_with_input(element, value)
        except:
            pass

        
def submit_form(form):
    # submit = driver.find_element(By.XPATH, f'{form_xpath}//*[@type="submit"]')
    # driver.find_element(By.TAG_NAME, 'body').click()
    submit = find_button()
    
    for _ in range(3):
        try:
            interact_with_input(submit, True)
            time.sleep(0.5)
        except:
            break

In [13]:
passing_set = {
    element_id: values[element_id]['passing'] for element_id in values.keys()
}


for element_id, element_values in values.items():
    get_to_form(driver)
    time.sleep(1)
    
    try:
        element = get_element(element_id)
    except:
        continue
    element_type = element.get_attribute('type') or 'text'
    
    if element_type in ['radio', 'checkbox', 'submit']:
        continue
    
    # print(element)
    
    for failing_value in element_values['failing']:
        # try:
        get_to_form(driver)
        time.sleep(1)
    
        passing_copy = copy.copy(passing_set)
        passing_copy[element_id] = failing_value
        send_values(values.keys(), passing_copy)

        time.sleep(1)

        submit_form(form)

        new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
        global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

        history_table.add(
            passing_copy,
            'base',
            global_feedback,
            driver.current_url
        )
        # except Exception as e:
        #     print(e)

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({
/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({
/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({
/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWa

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({
/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({
/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({
/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWa

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({
/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({
/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({
/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWa

In [14]:
for i, v in enumerate(history_table.table['values']):
    jv = json.loads(v)
    for key, value in jv.items():
        if key not in history_table.table.columns:
            history_table.table[key] = None
        history_table.table.loc[i, key] = value

In [19]:
history_table.table

,values,variation_type,feedback,new_url,type,name,category_id,description,sale_information,purchase_information,sale_price,purchase_price,tax_ids
0,"{""type"": ""product"", ""name"": """", ""category_id"":...",base,New Item\n \n\n\n\n\ngrade\n\n\n Add to ...,http://localhost:8080/1/common/items/create,product,,5,This is a test product,sale,sale,100,80,
1,"{""type"": ""product"", ""name"": ""123"", ""category_i...",base,New Item\n \n\n\n\n\ngrade\n\n\n Add to ...,http://localhost:8080/1/common/items/create,product,123,5,This is a test product,sale,sale,100,80,
2,"{""type"": ""product"", ""name"": ""Test Product 1"", ...",base,New Item\n \n\n\n\n\ngrade\n\n\n Add to ...,http://localhost:8080/1/common/items/create,product,Test Product 1,5,This is a test product,sale,sale,100,80,
3,"{""type"": ""product"", ""name"": ""Test Product 2"", ...",base,New Item\n \n\n\n\n\ngrade\n\n\n Add to ...,http://localhost:8080/1/common/items/create,product,Test Product 2,5,This is a test product,sale,sale,100,80,
4,"{""type"": ""product"", ""name"": ""Test Product"", ""c...",base,New Item\n \n\n\n\n\ngrade\n\n\n Add to ...,http://localhost:8080/1/common/items/create,product,Test Product,,This is a test product,sale,sale,100,80,
5,"{""type"": ""product"", ""name"": ""Test Product"", ""c...",base,New Item\n \n\n\n\n\ngrade\n\n\n Add to ...,http://localhost:8080/1/common/items/create,product,Test Product,123,This is a test product,sale,sale,100,80,
6,"{""type"": ""product"", ""name"": ""Test Product"", ""c...",base,New Item\n \n\n\n\n\ngrade\n\n\n Add to ...,http://localhost:8080/1/common/items/create,product,Test Product,General,This is a test product,sale,sale,100,80,
7,"{""type"": ""product"", ""name"": ""Test Product"", ""c...",base,New Item\n \n\n\n\n\ngrade\n\n\n Add to ...,http://localhost:8080/1/common/items/create,product,Test Product,New Category,This is a test product,sale,sale,100,80,
8,"{""type"": ""product"", ""name"": ""Test Product"", ""c...",base,New Item\n \n\n\n\n\ngrade\n\n\n Add to ...,http://localhost:8080/1/common/items/create,product,Test Product,5,,sale,sale,100,80,
9,"{""type"": ""product"", ""name"": ""Test Product"", ""c...",base,New Item\n \n\n\n\n\ngrade\n\n\n Add to ...,http://localhost:8080/1/common/items/create,product,Test Product,5,123,sale,sale,100,80,
